<a href="https://colab.research.google.com/github/RTae/AI102/blob/master/week2/AI102_Week2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes (Classification)

- Continuous

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
def NB_create_MeanStd_Storage(Feature_Name, All_Class):
  n_feature = len(Feature_Name)
  n_rows = n_feature+1
  n_class = len(All_Class)
  n_columns = n_class+1
  mean = np.zeros([n_rows,n_columns], dtype='object')
  std = np.zeros([n_rows,n_columns], dtype='object')
  for c in range(n_class):
    mean[0,c+1] = All_Class[c]
    std[0,c+1] = All_Class[c]
  for r in range(n_feature):
    mean[r+1,0] = Feature_Name[r]
    std[r+1,0] = Feature_Name[r]
  return mean,std

In [0]:
def NBCD_fit(X_Train, Y_Train, Feature_Name, All_Class):
  mean,std = NB_create_MeanStd_Storage(Feature_Name, All_Class)
  n_feature = len(Feature_Name)
  n_class = len(All_Class)
  for c, _class in enumerate(All_Class):
    filter_class = np.argwhere(Y_Train[:,0] == _class).ravel()
    X_Class = X_Train[filter_class]
    for r in range(n_feature):
      mean[r+1,c+1] = X_Class[:,r].mean()
      std[r+1,c+1] = X_Class[:,r].std()
  return mean,std

In [0]:
def normal_pdf(x, mean, std):
    degree = ((x - mean)/std)**2
    fraction = np.e**(-degree/2)
    denorminator = std*np.sqrt(2*np.pi)
    pdf = fraction/denorminator
    return pdf

In [0]:
def NBCD_predict(X_Test, mean, std, All_Class, Count_All_Class, top = 1):
  prob_class = Count_All_Class/(Count_All_Class.sum())
  n_feature = X_Test.shape[1]
  n_class = len(All_Class)
  Yhat = []
  for x_test in X_Test:
    prob_fgc = np.zeros([n_feature,n_class])
    for d in range(n_feature):
      for c in range(n_class):
        prob_fgc[d,c] = normal_pdf(x_test[d],mean[d+1,c+1],std[d+1,c+1])
    prob_afgc = prob_fgc.prod(axis = 0)
    prob_nb = prob_class*prob_afgc
    sorted_class = All_Class[prob_nb.argsort()[::-1]]
    Yhat.append(sorted_class)
  return np.array(Yhat)[:,:top]

In [0]:
def find_error_classification(Y, Yhat):
    N = Y.shape[0]
    error = (100/N)*(Y != Yhat).sum()
    return error

- Example I EURUSD

In [11]:
# Prepare data

from google.colab import files
uploaded = files.upload()

Saving EURUSD.xlsx to EURUSD.xlsx


In [0]:
Data = pd.read_excel('EURUSD.xlsx', usecols = 'AN:AX', skiprows = range(1, 7))

In [14]:
Data

,Delta 5,DD 5,Delta 4,DD 4,Delta 3,DD 3,Delta 2,DD 2,Delta 1,DD 1,Trend
0,0.01610,0.01120,-0.00650,-0.02260,0.01391,0.02041,-0.00055,-0.01446,0.00539,0.00594,-1
1,-0.00650,-0.02260,0.01391,0.02041,-0.00055,-0.01446,0.00539,0.00594,-0.00237,-0.00776,-1
2,0.01391,0.02041,-0.00055,-0.01446,0.00539,0.00594,-0.00237,-0.00776,-0.00180,0.00057,-1
3,-0.00055,-0.01446,0.00539,0.00594,-0.00237,-0.00776,-0.00180,0.00057,-0.01066,-0.00886,1
4,0.00539,0.00594,-0.00237,-0.00776,-0.00180,0.00057,-0.01066,-0.00886,0.01930,0.02996,1
...,...,...,...,...,...,...,...,...,...,...,...
4121,0.00228,0.00232,0.00318,0.00090,-0.00194,-0.00512,0.00013,0.00207,-0.00073,-0.00086,-1
4122,0.00318,0.00090,-0.00194,-0.00512,0.00013,0.00207,-0.00073,-0.00086,-0.00417,-0.00344,-1
4123,-0.00194,-0.00512,0.00013,0.00207,-0.00073,-0.00086,-0.00417,-0.00344,-0.00321,0.00096,-1
4124,0.00013,0.00207,-0.00073,-0.00086,-0.00417,-0.00344,-0.00321,0.00096,-0.00011,0.00310,-1


In [0]:
DataMatrix = Data.values

In [16]:
DataMatrix.shape

(4126, 11)

In [0]:
D = DataMatrix.shape[1] - 1
X = DataMatrix[:, :D]
Y = DataMatrix[:, D:]

In [0]:
start_train = 0
end_train = -450
end_test = -300

X_Train = X[start_train:end_train, :]
Y_Train = Y[start_train:end_train, :]

X_Test = X[end_train:end_test, :]
Y_Test = Y[end_train:end_test, :]

In [0]:
# Learning

Feature_Name = np.array(Data.columns[:-1])
All_Class, Count_All_Class = np.unique(Y_Train[:, :], return_counts = True)

In [0]:
mean, std = NBCD_fit(X_Train, Y_Train, Feature_Name, All_Class)

In [0]:
# Predict

Yhat_Test = NBCD_predict(X_Test, mean, std, All_Class, Count_All_Class)

In [43]:
Yhat_Test

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
      

In [0]:
error_Test = find_error_classification(Y_Test, Yhat_Test)

In [45]:
error_Test

39.33333333333333

- Example II BMI

In [46]:
# Prepare data

from google.colab import files
uploaded = files.upload()

Saving BMI_Dataset_Classification.xlsx to BMI_Dataset_Classification.xlsx


In [0]:
Data = pd.read_excel('BMI_Dataset_Classification.xlsx', usecols = 'A:C')

In [49]:
Data

,Weight,Height,Target
0,66.17,185.21,normal
1,71.27,168.91,fat lv3
2,73.17,179.83,normal
3,81.74,171.76,fat lv2
4,92.97,172.89,fat lv3
...,...,...,...
995,63.24,141.45,fat lv3
996,47.55,166.64,thin
997,68.10,168.52,fat lv1
998,74.39,169.24,fat lv2


In [0]:
DataMatrix = Data.values

In [51]:
DataMatrix.shape

(1000, 3)

In [0]:
D = DataMatrix.shape[1] - 1

X = DataMatrix[:, :D]
Y = DataMatrix[:, D:]

In [0]:
start_train = 0
end_train = -200

X_Train = X[start_train:end_train, :]
Y_Train = Y[start_train:end_train, :]

X_Test = X[end_train:, :]
Y_Test = Y[end_train:, :]

In [0]:
# Learning 

Feature_Name = np.array(Data.columns[:-1])
All_Class, Count_All_Class = np.unique(Y_Train[:, :], return_counts = True)

In [0]:
mean, std = NBCD_fit(X_Train, Y_Train, Feature_Name, All_Class)

In [0]:
# Predict

Yhat_Test = NBCD_predict(X_Test, mean, std, All_Class, Count_All_Class)

In [58]:
Yhat_Test

array([['fat lv1'],
       ['fat lv2'],
       ['fat lv1'],
       ['fat lv3'],
       ['fat lv2'],
       ['fat lv3'],
       ['normal'],
       ['fat lv1'],
       ['normal'],
       ['fat lv2'],
       ['fat lv2'],
       ['fat lv2'],
       ['fat lv2'],
       ['fat lv3'],
       ['normal'],
       ['fat lv2'],
       ['thin'],
       ['normal'],
       ['fat lv2'],
       ['normal'],
       ['normal'],
       ['fat lv2'],
       ['fat lv1'],
       ['fat lv1'],
       ['normal'],
       ['thin'],
       ['normal'],
       ['normal'],
       ['fat lv2'],
       ['normal'],
       ['normal'],
       ['fat lv2'],
       ['fat lv2'],
       ['fat lv2'],
       ['normal'],
       ['fat lv2'],
       ['normal'],
       ['normal'],
       ['fat lv2'],
       ['normal'],
       ['fat lv2'],
       ['normal'],
       ['fat lv2'],
       ['fat lv3'],
       ['normal'],
       ['normal'],
       ['fat lv3'],
       ['fat lv3'],
       ['normal'],
       ['fat lv3'],
       ['fat lv2'],
     

In [0]:
error_Test = find_error_classification(Y_Test, Yhat_Test)

In [60]:
error_Test

28.5

- Discrete

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
def NB_find_Feature_Value(X_Train, Feature_Name):
  Feture_Value = []
  D = len(Feature_Name)
  for d in range(D):
    Value = np.unique(X_Train[:,d])
    Feture_Value.append(Value)
  return Feture_Value

In [0]:
def NB_create_DictStorage(Feture_name,Feature_Value,All_Class):
  DictStorge = {}
  n_class = len(All_Class)
  n_colums = n_class+1
  D = len(Feture_name)
  for d in range(D):
    fn = Feture_name[d]
    n_value = len(Feature_Value[d])
    n_rows = n_value + 1
    MaxtrixStorge = np.zeros([n_rows,n_colums],dtype = 'object')
    for c in range(n_class):
      MaxtrixStorge[0,c+1] = All_Class[c]
    for r in range(n_value):
      MaxtrixStorge[r+1,0] = Feature_Value[d][r]
    DictStorge[fn] = np.array(MaxtrixStorge)
  return DictStorge

In [0]:
def NB_count_element(X_Train,Y_Train,Feature_Name,empty_Count):
  D = len(Feature_Name)
  for d in range(D):
    fn = Feature_Name[d]
    n_rows, n_columns = empty_Count[fn].shape
    for c in range(1,n_columns):
      filter_class = (Y_Train[:,0] == empty_Count[fn][0,c])
      for r in range(1,n_rows):
        filter_feature = (X_Train[:,d] == empty_Count[fn][r,0])
        empty_Count[fn][r,c] = (filter_feature & filter_class).sum()
  return empty_Count

In [0]:
def NB_calculate_prob(empty_Prob,Count_Martix,Count_all_CLass,Feature_Name):
  D = len(Feature_Name)
  for d in range(D):
    fn = Feature_Name[d]
    empty_Prob[fn][1:,1:] = Count_Martix[fn][1:,1:]/Count_all_CLass
  return empty_Prob


In [0]:
def NBDD_fit(X_train,Y_train,Feature_Name,All_Class,Count_all_Class):
  Feture_Value = NB_find_Feature_Value(X_train,Feature_Name)
  empty_Count = NB_create_DictStorage(Feature_Name,Feture_Value,All_Class)
  empty_Prob = NB_create_DictStorage(Feature_Name,Feture_Value,All_Class)
  Count_Martix = NB_count_element(X_train,Y_train,Feature_Name,empty_Count)
  Prob_Martix = NB_calculate_prob(empty_Prob,Count_Martix,Count_all_Class,Feature_Name)

  return Count_Martix,Prob_Martix

In [0]:
def NBDD_predict(X_Test, Prob_Martix, Feature_Name, Feature_Value, All_Class, Count_all_Class, top = 1):
  prob_class = Count_all_Class/(Count_all_Class.sum())
  Yhat = []
  D = len(Feature_Value)
  C = len(All_Class)
  for x_test in X_Test:
    prob_fgc = np.zeros([D,C])
    for d in range(D):
      fn = Feature_Name[d]
      r = np.argwhere(Feature_Value[d] == x_test[d]).ravel()[0]
      prob_fgc[d,:] = Prob_Martix[fn][r+1,1:]
    prob_afgc = prob_fgc.prod(axis=0)
    prob_nb = (prob_class*prob_afgc)
    sorted_class = All_Class[prob_nb.argsort()[::-1]]
    Yhat.append(sorted_class)
  return np.array(Yhat)[:,:top]

In [0]:
def find_error_classification(Y, Yhat):
    N = Y.shape[0]
    error = (100/N)*(Y != Yhat).sum()
    return error

In [0]:
FN = np.array(['Work','Education'])
AC = np.array(['poor','rich'])
Count_all_CLass = np.array([3,3])
X_train = np.array([['Work hard','study'],['Work hard','not study'],['lazy','study'],['lazy','not study'],['Work hard','not study'],['lazy','not study']])
Y_train = np.array([['rich'],['rich'],['rich'],['poor'],['poor'],['poor']])
X_Test = np.array([['Work hard','study']])

In [0]:
FV = NB_find_Feature_Value(X_train,FN)
Count_Martix, Prob_Martix = NBDD_fit(X_train,Y_train,FN,AC,Count_all_CLass)
Yhat = NBDD_predict(X_Test,Prob_Martix,FN,FV,AC,Count_all_CLass)
print(Yhat)

[['poor']]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]


- Example I EURUSD

In [0]:
# Prepare data

from google.colab import files
uploaded = files.upload()

In [0]:
Data = pd.read_excel('EURUSD_J_Patel.xlsx',usecols = 'B:L')

In [0]:
Data

,SMA,WMA,STOCH_SLOWK,STOCH_SLOWD,WILLR,MACD,RSI,CCI,AD,MOM,Trend
0,0,0,0,0,0,0,1,0,1,1,0
1,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1,0,0,1
3,0,0,0,0,1,0,1,1,1,0,1
4,0,0,1,0,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1668,1,1,0,1,1,1,1,0,1,1,1
1669,1,1,1,1,1,1,1,1,0,1,0
1670,1,1,1,1,0,1,0,0,0,1,1
1671,1,1,1,1,1,1,1,0,1,1,0


In [0]:
Data.shape

(1673, 11)

In [0]:
DataMatrix = Data.values
D = DataMatrix.shape[1]-1
X = DataMatrix[:,:D]
Y = DataMatrix[:,D:]

In [0]:
start_train = 0
end_train = -150

X_Train =  X[start_train:end_train,:]
Y_Train =  Y[start_train:end_train,:]

X_Test = X[end_train:, :]
Y_Test = Y[end_train:, :]

In [0]:
Feature_name = np.array(Data.columns[:-1])
Feature_Value = NB_find_Feature_Value(X_Train,Feture_name)
All_Class, Count_All_Class = np.unique(Y_Train[:,:],return_counts=True)

In [0]:
# Learning

Count_Martix, Prob_Martix = NBDD_fit(X_Train,Y_Train,Feature_name,All_Class,Count_All_Class)

In [0]:
# Predict

Yhat_Test = NBDD_predict(X_Test, Prob_Martix, Feature_name, Feature_Value, All_Class, Count_All_Class, top=1)
error = find_error_classification(Y_Test, Yhat_Test)
print(error)

53.33333333333333


- Example II Men or Women

In [0]:
# Prepare data

uploaded = files.upload()

Saving Men_or_Women.xlsx to Men_or_Women (1).xlsx


In [0]:
Data = pd.read_excel('Men_or_Women.xlsx')

In [0]:
Data

,hair_style,sound_style,has_earring,Class
0,short,low-pitched,yes,men
1,long,high-pitched,yes,women
2,long,low-pitched,yes,women
3,long,low-pitched,no,men
4,long,high-pitched,yes,women
...,...,...,...,...
2032,long,high-pitched,yes,women
2033,long,high-pitched,yes,women
2034,long,low-pitched,yes,women
2035,short,low-pitched,no,men


In [0]:
Data.shape

(2037, 4)

In [0]:
DataMatrix = Data.values
D = DataMatrix.shape[1]-1
X = DataMatrix[:,:D]
Y = DataMatrix[:,D:]

In [0]:
start_train = 0
end_train = -150

X_Train =  X[start_train:end_train,:]
Y_Train =  Y[start_train:end_train,:]

X_Test = X[end_train:, :]
Y_Test = Y[end_train:, :]

In [0]:
Feature_name = np.array(Data.columns[:-1])
Feature_Value = NB_find_Feature_Value(X_Train,Feature_name)
All_Class, Count_All_Class = np.unique(Y_Train[:,:],return_counts=True)

In [0]:
# Learning

Count_Martix, Prob_Martix = NBDD_fit(X_Train,Y_Train,Feature_name,All_Class,Count_All_Class)

In [0]:
# Predict

Yhat_Test = NBDD_predict(X_Test, Prob_Martix, Feature_name, Feature_Value, All_Class, Count_All_Class, top=1)
index = np.argwhere(Y_Test[:, 0] != Yhat_Test[:, 0]).ravel()

In [0]:
np.hstack([Y_Test[index], Yhat_Test[index]])

array([['women', 'men'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['men', 'women'],
       ['men', 'women'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['women', 'men'],
       ['women', 'men'],
       ['women', 'men'],
       ['men', 'women'],
       ['men', 'women'],
       ['men', 'women'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['men', 'women'],
       ['men', 'women'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['men', 'women'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],


In [0]:
X_Test[index]

array([['long', 'high-pitched', 'yes'],
       ['short', 'low-pitched', 'no'],
       ['long', 'high-pitched', 'yes'],
       ['short', 'high-pitched', 'no'],
       ['long', 'low-pitched', 'yes'],
       ['short', 'low-pitched', 'no'],
       ['short', 'low-pitched', 'no'],
       ['short', 'low-pitched', 'no'],
       ['short', 'low-pitched', 'no'],
       ['short', 'high-pitched', 'yes'],
       ['short', 'low-pitched', 'yes'],
       ['long', 'high-pitched', 'yes'],
       ['short', 'low-pitched', 'yes'],
       ['long', 'low-pitched', 'yes'],
       ['short', 'low-pitched', 'yes'],
       ['short', 'high-pitched', 'yes'],
       ['long', 'high-pitched', 'yes'],
       ['short', 'high-pitched', 'yes'],
       ['short', 'low-pitched', 'no'],
       ['short', 'high-pitched', 'no'],
       ['short', 'low-pitched', 'yes'],
       ['short', 'low-pitched', 'no'],
       ['short', 'high-pitched', 'yes'],
       ['short', 'high-pitched', 'no'],
       ['short', 'low-pitched', 'no'],
      

In [0]:
np.hstack([Y_Test, Yhat_Test])

array([['women', 'men'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['men', 'women'],
       ['men', 'women'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['women', 'men'],
       ['women', 'men'],
       ['women', 'men'],
       ['men', 'women'],
       ['men', 'women'],
       ['men', 'women'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['men', 'women'],
       ['men', 'women'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['men', 'women'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],
       ['women', 'men'],
       ['men', 'women'],


In [0]:
error = find_error_classification(Y_Test, Yhat_Test)
print(error)

97.33333333333333


- Example III Lotto

In [0]:
# Prepare data

uploaded = files.upload()

Saving Lotto_Dataset.xlsx to Lotto_Dataset.xlsx


In [0]:
Feature_DF = pd.read_excel('Lotto_Dataset.xlsx', usecols = 'D:G')

Up100_DF = pd.read_excel('Lotto_Dataset.xlsx', usecols = 'H')
Up10_DF = pd.read_excel('Lotto_Dataset.xlsx', usecols = 'I')
Up1_DF = pd.read_excel('Lotto_Dataset.xlsx', usecols = 'J')

In [0]:
Feature = Feature_DF.values

Up100 = Up100_DF.values
Up10 = Up10_DF.values
Up1 = Up1_DF.values

In [0]:
start_train = 0
end_train = -100

Feature_Train = Feature[:end_train, :]
Up100_Train = Up100[:end_train, :]
Up10_Train = Up10[:end_train, :]
Up1_Train = Up1[:end_train, :]

Feature_Test = Feature[end_train:, :]
Up100_Test = Up100[end_train:, :]
Up10_Test = Up10[end_train:, :]
Up1_Test = Up1[end_train:, :]

In [0]:
Feature_Name = np.array(Feature_DF.columns[:])
Feature_Value = NB_find_Feature_Value(Feature_Train, Feature_Name)
All_Class, Count_All_Class = np.unique(Up100_Train[:, :], return_counts = True)


In [0]:
# Learning

Count_Matrix_Up100, Prob_Matrix_Up100 = NBDD_fit(Feature_Train, Up100_Train, Feature_Name, All_Class, Count_All_Class)
Count_Matrix_Up100, Prob_Matrix_Up100 = NBDD_fit(Feature_Train, Up100_Train, Feature_Name, All_Class, Count_All_Class)
Count_Matrix_Up1, Prob_Matrix_Up1 = NBDD_fit(Feature_Train, Up1_Train, Feature_Name, All_Class, Count_All_Class)

In [0]:
Feature_Test = np.array([['Friday', -1, 'Ninth', 'Pig']], dtype='object')

In [0]:
Feature_Test

array([['Friday', -1, 'Ninth', 'Pig']], dtype=object)

In [0]:
Up10hat_Test = NBDD_predict(Feature_Test, Prob_Matrix_Up10, Feature_Name, Feature_Value, All_Class, Count_All_Class, top=5)
Up1hat_Test = NBDD_predict(Feature_Test, Prob_Matrix_Up1, Feature_Name, Feature_Value, All_Class, Count_All_Class, top=5)

In [0]:
print(Up10hat_Test)
print(Up1hat_Test)

[[5 9 1 6 2]]
[[7 6 1 8 2]]
